In [1]:

# import os
# import requests  # Make sure requests is installed
# import json
# import numpy as np
# import tensorflow as tf
# from tqdm import tqdm

# def download_and_load_gpt2(model_size, models_dir):
#     # Validate model size
#     allowed_sizes = ("124M", "355M", "774M", "1558M")
#     if model_size not in allowed_sizes:
#         raise ValueError(f"Model size not in {allowed_sizes}")

#     # Define paths
#     model_dir = os.path.join(models_dir, model_size)
#     base_url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
#     filenames = [
#         "checkpoint", "encoder.json", "hparams.json",
#         "model.ckpt.data-00000-of-00001", "model.ckpt.index",
#         "model.ckpt.meta", "vocab.bpe"
#     ]

#     # Download files
#     os.makedirs(model_dir, exist_ok=True)
#     for filename in filenames:
#         file_url = os.path.join(base_url, model_size, filename)
#         file_path = os.path.join(model_dir, filename)
#         download_file(file_url, file_path)

#     ## We have reached here until now ---> we have downloaded the files on our local machine.

#     # Load settings and params
#     tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
#     settings = json.load(open(os.path.join(model_dir, "hparams.json")))
#     params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, settings)

#     return settings, params

# def download_file(url, destination):
#     try:
#         # Send a GET request to download the file, disabling SSL verification
#         response = requests.get(url, stream=True, verify=False)

#         # Get the total file size from headers, defaulting to 0 if not present
#         file_size = int(response.headers.get("content-length", 0))

#         # Check if file exists and has the same size
#         if os.path.exists(destination):
#             file_size_local = os.path.getsize(destination)
#             if file_size == file_size_local:
#                 print(f"File already exists and is up-to-date: {destination}")
#                 return

#         # Define the block size for reading the file
#         block_size = 1024  # 1 Kilobyte

#         # Initialize the progress bar with total file size
#         progress_bar_description = url.split("/")[-1]  # Extract filename from URL
#         with tqdm(total=file_size, unit="iB", unit_scale=True, desc=progress_bar_description) as progress_bar:
#             # Open the destination file in binary write mode
#             with open(destination, "wb") as file:
#                 # Iterate over the file data in chunks
#                 for chunk in response.iter_content(block_size):
#                     progress_bar.update(len(chunk))  # Update progress bar
#                     file.write(chunk)  # Write the chunk to the file

#     except requests.exceptions.RequestException as e:
#         print(f"Error downloading the file: {e}")
#         print(f"Please check the URL: {url}")

# def load_gpt2_params_from_tf_ckpt(ckpt_path, settings):
#     # Initialize parameters dictionary with empty blocks for each layer
#     params = {"blocks": [{} for _ in range(settings["n_layer"])]}

#     # Iterate over each variable in the checkpoint
#     for name, _ in tf.train.list_variables(ckpt_path):
#         # Load the variable and remove singleton dimensions
#         variable_array = np.squeeze(tf.train.load_variable(ckpt_path, name))

#         # Process the variable name to extract relevant parts
#         variable_name_parts = name.split("/")[1:]  # Skip the 'model/' prefix

#         # Identify the target dictionary for the variable
#         target_dict = params
#         if variable_name_parts[0].startswith("h"):
#             layer_number = int(variable_name_parts[0][1:])
#             target_dict = params["blocks"][layer_number]

#         # Recursively access or create nested dictionaries
#         for key in variable_name_parts[1:-1]:
#             target_dict = target_dict.setdefault(key, {})

#         # Assign the variable array to the last key
#         last_key = variable_name_parts[-1]
#         target_dict[last_key] = variable_array

#     return params


In [2]:
import os
import json
import numpy as np
import tensorflow as tf

def load_gpt2(model_size, models_dir):
    """
    Load GPT-2 model settings and parameters from local directory.

    Args:
        model_size (str): One of ("124M", "355M", "774M", "1558M")
        models_dir (str): Path to the folder where models are stored

    Returns:
        settings (dict): Model hyperparameters
        params (dict): Model weights organized by layer
    """
    # Paths
    model_dir = os.path.join(models_dir, model_size)
    tf_ckpt_path = tf.train.latest_checkpoint(model_dir)

    if tf_ckpt_path is None:
        raise FileNotFoundError(f"No TensorFlow checkpoint found in {model_dir}")

    # Load settings
    settings = json.load(open(os.path.join(model_dir, "hparams.json")))

    # Load parameters from checkpoint
    params = {"blocks": [{} for _ in range(settings["n_layer"])]}
    for name, _ in tf.train.list_variables(tf_ckpt_path):
        variable_array = np.squeeze(tf.train.load_variable(tf_ckpt_path, name))
        variable_name_parts = name.split("/")[1:]  # skip 'model/'
        target_dict = params

        if variable_name_parts[0].startswith("h"):  # layer params
            layer_number = int(variable_name_parts[0][1:])
            target_dict = params["blocks"][layer_number]

        # Go inside nested dicts
        for key in variable_name_parts[1:-1]:
            target_dict = target_dict.setdefault(key, {})

        # Final assignment
        last_key = variable_name_parts[-1]
        target_dict[last_key] = variable_array

    return settings, params


# --------------------------
# Example usage
# --------------------------
model_size = "124M"  # or "355M", "774M", "1558M"
models_dir = "/content/drive/MyDrive/"  # where you saved the files

settings, params = load_gpt2(model_size, models_dir)

print("Model settings:", settings)
print("Top-level keys in params:", params.keys())
print("Number of layers:", len(params["blocks"]))


Model settings: {'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}
Top-level keys in params: dict_keys(['blocks', 'b', 'g', 'wpe', 'wte'])
Number of layers: 12


In [3]:
import pandas as pd
import tiktoken
tokenizer= tiktoken.get_encoding("gpt2")
df = pd.read_csv(
    "/content/drive/MyDrive/Email-classification-dataset-for-LLM/SMSSpamCollection",
    sep="\t",
    header=None,
    names=["Label", "Text"]
)
df.head(5)


,Label,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
print(df['Label'].value_counts())

Label
ham     4825
spam     747
Name: count, dtype: int64


In [5]:
num_spam= df[df['Label'] == "spam"].shape[0]
ham_subst= df[df['Label']== "ham"].sample(num_spam,random_state=123)
balance_df= pd.concat([ham_subst,df[df['Label'] == "spam"]])
balance_df.shape


(1494, 2)

In [6]:
balance_df['Label'] = balance_df['Label'].map({'ham':0,'spam':1})
balance_df['Label']

,Label
4307,0
4138,0
4831,0
4461,0
5440,0
...,...
5537,1
5540,1
5547,1
5566,1


In [7]:
balance_df = balance_df.sample(frac=1, random_state=123).reset_index(drop=True)

train_end = int(len(balance_df.Label) * 0.7)
val_end = train_end + int(len(balance_df.Label) * 0.1)

train_df = balance_df[:train_end]
validation_df = balance_df[train_end:val_end]
test_df = balance_df[val_end:]

train_df.to_csv("train.csv", index=None)
validation_df.to_csv("validation.csv", index=None)
test_df.to_csv("test.csv", index=None)

print(train_df.head(5))
print(test_df.head(5))
print(validation_df.head(5))

   Label                                               Text
0      0                Dude how do you like the buff wind.
1      0  Tessy..pls do me a favor. Pls convey my birthd...
2      1  Reminder: You have not downloaded the content ...
3      1  Got what it takes 2 take part in the WRC Rally...
4      1  Shop till u Drop, IS IT YOU, either 10K, 5K, £...
      Label                                               Text
1194      1                     85233 FREE>Ringtone!Reply REAL
1195      1  Ur cash-balance is currently 500 pounds - to m...
1196      1  Thanks for your ringtone order, reference numb...
1197      0                We live in the next  &lt;#&gt; mins
1198      1  1st wk FREE! Gr8 tones str8 2 u each wk. Txt N...
      Label                                               Text
1045      1  Mila, age23, blonde, new in UK. I look sex wit...
1046      1  Hungry gay guys feeling hungry and up 4 it, no...
1047      0  Ugh. Gotta drive back to sd from la. My butt i...
1048      

In [8]:
import torch
from torch.utils.data import Dataset


class SpamDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50256):
        self.data = pd.read_csv(csv_file)

        # Pre-tokenize texts
        self.encoded_texts = [
            tokenizer.encode(text) for text in self.data["Text"]
        ]

        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length

            # Truncate sequences if they are longer than max_length
            self.encoded_texts = [
                encoded_text[:self.max_length]
                for encoded_text in self.encoded_texts
            ]

        # Pad sequences to the longest sequence
        self.encoded_texts = [
            encoded_text + [pad_token_id] * (self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts
        ]

    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        label = self.data.iloc[index]["Label"]
        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        max_length = 0
        for encoded_text in self.encoded_texts:
            encoded_length = len(encoded_text)
            if encoded_length > max_length:
                max_length = encoded_length
        return max_length

In [9]:
train_dataset= SpamDataset(csv_file="train.csv",max_length=None,tokenizer=tokenizer)
test_dataset= SpamDataset(csv_file="test.csv",max_length=None,tokenizer=tokenizer)
val_dataset=SpamDataset(csv_file="validation.csv",max_length=None,tokenizer=tokenizer)

df= pd.read_csv("test.csv")
df.head()

,Label,Text
0,1,85233 FREE>Ringtone!Reply REAL
1,1,Ur cash-balance is currently 500 pounds - to m...
2,1,"Thanks for your ringtone order, reference numb..."
3,0,We live in the next &lt;#&gt; mins
4,1,1st wk FREE! Gr8 tones str8 2 u each wk. Txt N...


In [10]:
from torch.utils.data import DataLoader

num_workers= 0
batch_size=8
train_loader=DataLoader(
    dataset=train_dataset,
    shuffle=True,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=True
)
test_loader=DataLoader(
    dataset=test_dataset,
    shuffle=True,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False
)
val_loader=DataLoader(
    dataset=val_dataset,
    shuffle=True,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False
)

In [11]:
import torch
import torch.nn as nn
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]), ## Expansion
            GELU(), ## Activation
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]), ## Contraction
        )

    def forward(self, x):
        return self.layers(x)


class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        # 2*4*768
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x
        # 2*4*768

class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


In [12]:
def assign(left,right):
  if left.shape != right.shape:
    raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")
  return torch.nn.Parameter(torch.tensor(right))

import numpy as np

def load_weights_into_gpt(gpt, params):
    gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe'])
    gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])

    for b in range(len(params["blocks"])):
        q_w, k_w, v_w = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.weight = assign( gpt.trf_blocks[b].att.W_query.weight, q_w.T)
        gpt.trf_blocks[b].att.W_key.weight = assign( gpt.trf_blocks[b].att.W_key.weight, k_w.T)
        gpt.trf_blocks[b].att.W_value.weight = assign(gpt.trf_blocks[b].att.W_value.weight, v_w.T)

        q_b, k_b, v_b = np.split((params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.bias = assign( gpt.trf_blocks[b].att.W_query.bias, q_b)
        gpt.trf_blocks[b].att.W_key.bias = assign( gpt.trf_blocks[b].att.W_key.bias, k_b)
        gpt.trf_blocks[b].att.W_value.bias = assign(gpt.trf_blocks[b].att.W_value.bias, v_b)

        gpt.trf_blocks[b].att.out_proj.weight = assign(gpt.trf_blocks[b].att.out_proj.weight,params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].att.out_proj.bias = assign( gpt.trf_blocks[b].att.out_proj.bias, params["blocks"][b]["attn"]["c_proj"]["b"])

        gpt.trf_blocks[b].ff.layers[0].weight = assign( gpt.trf_blocks[b].ff.layers[0].weight, params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign( gpt.trf_blocks[b].ff.layers[0].bias, params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.trf_blocks[b].ff.layers[2].weight = assign( gpt.trf_blocks[b].ff.layers[2].weight, params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign( gpt.trf_blocks[b].ff.layers[2].bias,params["blocks"][b]["mlp"]["c_proj"]["b"])

        gpt.trf_blocks[b].norm1.scale = assign( gpt.trf_blocks[b].norm1.scale, params["blocks"][b]["ln_1"]["g"])
        gpt.trf_blocks[b].norm1.shift = assign( gpt.trf_blocks[b].norm1.shift,params["blocks"][b]["ln_1"]["b"])
        gpt.trf_blocks[b].norm2.scale = assign( gpt.trf_blocks[b].norm2.scale,params["blocks"][b]["ln_2"]["g"])
        gpt.trf_blocks[b].norm2.shift = assign( gpt.trf_blocks[b].norm2.shift, params["blocks"][b]["ln_2"]["b"])

    gpt.final_norm.scale = assign(gpt.final_norm.scale, params["g"])
    gpt.final_norm.shift = assign(gpt.final_norm.shift, params["b"])
    gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"])

def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):

    # For-loop is the same as before: Get logits, and only focus on last time step
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]

        # New: Filter logits with top_k sampling
        if top_k is not None:
            # Keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float("-inf")).to(logits.device), logits)

        # New: Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # Apply softmax to get probabilities
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)

        # Otherwise same as before: get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

        if idx_next == eos_id:  # Stop generating early if end-of-sequence token is encountered and eos_id is specified
            break

        # Same as before: append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

    return idx

def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context

    ###Input batch:
 ###tensor([[6109, 3626, 6100,  345],
        ##[6109, 1110, 6622,  257]])

    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond) ### batch, n_tokens, vocab_size

        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx


def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

In [13]:
CHOOSE_MODEL = "gpt2-small (124M)"
INPUT_PROMPT = "Every effort moves"

BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

assert train_dataset.max_length <= BASE_CONFIG["context_length"], (
    f"Dataset length {train_dataset.max_length} exceeds model's context "
    f"length {BASE_CONFIG['context_length']}. Reinitialize data sets with "
    f"`max_length={BASE_CONFIG['context_length']}`"
)

In [14]:
# model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
# setting, params = download_and_load_gpt2(model_size=model_size,models_dir= "124M")

In [31]:
model =GPTModel(BASE_CONFIG)
load_weights_into_gpt(model,params)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()


GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=7

In [20]:
text_1 = "My name is"

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(text_1, tokenizer),
    max_new_tokens=25,
    context_size=BASE_CONFIG["context_length"]
)

print(token_ids_to_text(token_ids, tokenizer))

My name is John. I'm a man of God. I'm a man of God. I'm a man of God. I'm


In [21]:
for param in model.parameters():
  param.requires_grad = False # this parameters are freezed

In [24]:
torch.manual_seed(123)
num_class=2

# Setting output dimnesion 50257 ---> 2
model.out_head= torch.nn.Linear(in_features=BASE_CONFIG["emb_dim"],out_features=num_class)

#Now we finetue last transformer block
for parma in model.trf_blocks[-1].parameters():
  param.requires_grad = True
for param in model.final_norm.parameters():
  param.requires_grad= True


### CALCULATING CLASSIFICATION LOSS AND ACCURAY

In [25]:
def calc_accuracy_loader(data_loader,model,device,num_batches=None):
  model.eval()
  correct_prediction,num_examples= 0,0
  if num_batches is None:
    num_batches= len(data_loader)
  else:
    num_batches= min(num_batches,len(data_loader))
  for i,(input_batch,target_batch) in enumerate(data_loader):
    if i < num_batches:
      input_batch ,target_batch= input_batch.to(device),target_batch.to(device)

      with torch.no_grad():
        logits= model(input_batch)[:,-1,:]
        predicted_labels = torch.argmax(logits,dim=-1)

        num_examples += predicted_labels.shape[0]
        correct_prediction +=(predicted_labels == target_batch).sum().item()
    else:
      break
  return correct_prediction/ num_examples


In [26]:
def calc_loss_batch(input_batch,target_batch,model,device):
  input_batch, target_batch= input_batch.to(device), target_batch.to(device)
  logits = model(input_batch)[:,-1,:] #Logits of last output vector
  loss= torch.nn.functional.cross_entropy(logits,target_batch)
  return loss

In [27]:
def calc_loss_loader(data_loader,model,device,num_batches =None):
  total_loss= 0.
  if len(data_loader)==0:
    return float('nan')
  elif num_batches == None:
    num_bathces= len(data_loader)
  else:
    num_batches= min(num_batches,len(data_loader))

    for i,(input_batch,target_batch) in enumerate(data_loader):
      if i < num_batches:
        loss= calc_loss_batch(input_batch,target_batch,model,device)
        total_loss +=loss.item()
      else:
        break
  return total_loss / num_batches



## FINETUNING THE MODEL ON SUPERVISED DATA

Step 1: Set model to training mode

Step 2: Reset loss gradients from previous batch iteration

Step 3: Calculate loss gradients

Step 4: Update model weights using loss gradients

Step 5: New: track examples instead of tokens

Step 6: Optional evaluation step

Step 7: Calculate accuracy after each epoch

In [28]:
# Overall the same as `train_model_simple` in chapter 5
def train_classifier_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                            eval_freq, eval_iter):
    # Initialize lists to track losses and examples seen
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    examples_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        for input_batch, target_batch in train_loader:
            # ✅ Move data to device (GPU or CPU)
            input_batch = input_batch.to(device)
            target_batch = target_batch.to(device)

            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step()
            examples_seen += input_batch.shape[0] # New: track examples instead of tokens
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Calculate accuracy after each epoch
        train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=eval_iter)
        val_accuracy = calc_accuracy_loader(val_loader, model, device, num_batches=eval_iter)
        print(f"Training accuracy: {train_accuracy*100:.2f}% | ", end="")
        print(f"Validation accuracy: {val_accuracy*100:.2f}%")
        train_accs.append(train_accuracy)
        val_accs.append(val_accuracy)

    return train_losses, val_losses, train_accs, val_accs, examples_seen


In [29]:
# Same as chapter 5
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

In [32]:
import time

start_time = time.time()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(123)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.1)

num_epochs = 5
train_losses, val_losses, train_accs, val_accs, examples_seen = train_classifier_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=50, eval_iter=5,
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 3.917, Val loss 5.129
Ep 1 (Step 000050): Train loss 0.368, Val loss 0.303
Ep 1 (Step 000100): Train loss 0.049, Val loss 0.414
Training accuracy: 87.50% | Validation accuracy: 90.00%
Ep 2 (Step 000150): Train loss 0.221, Val loss 0.141
Ep 2 (Step 000200): Train loss 0.272, Val loss 0.065
Ep 2 (Step 000250): Train loss 0.125, Val loss 0.127
Training accuracy: 97.50% | Validation accuracy: 100.00%
Ep 3 (Step 000300): Train loss 0.014, Val loss 0.100
Ep 3 (Step 000350): Train loss 0.058, Val loss 0.094
Training accuracy: 100.00% | Validation accuracy: 97.50%
Ep 4 (Step 000400): Train loss 0.003, Val loss 0.001
Ep 4 (Step 000450): Train loss 0.002, Val loss 0.136
Ep 4 (Step 000500): Train loss 0.017, Val loss 0.209
Training accuracy: 100.00% | Validation accuracy: 97.50%
Ep 5 (Step 000550): Train loss 0.000, Val loss 0.001
Ep 5 (Step 000600): Train loss 0.009, Val loss 0.213
Training accuracy: 100.00% | Validation accuracy: 95.00%
Training completed in 3.48 

In [ ]:
def classify_review(text, model, tokenizer, device, max_length=None, pad_token_id=50256):
    model.eval()

    # Prepare inputs to the model
    input_ids = tokenizer.encode(text)
    supported_context_length = model.pos_emb.weight.shape[0]
    # Note: In the book, this was originally written as pos_emb.weight.shape[1] by mistake
    # It didn't break the code but would have caused unnecessary truncation (to 768 instead of 1024)

    # Truncate sequences if they too long
    input_ids = input_ids[:min(max_length, supported_context_length)]

    # Pad sequences to the longest sequence
    input_ids += [pad_token_id] * (max_length - len(input_ids))
    input_tensor = torch.tensor(input_ids, device=device).unsqueeze(0) # add batch dimension

    # Model inference
    with torch.no_grad():
        logits = model(input_tensor)[:, -1, :]  # Logits of the last output token
    predicted_label = torch.argmax(logits, dim=-1).item()

    # Return the classified result
    return "spam" if predicted_label == 1 else "not spam"

In [ ]:
text_1 = (
    "You are a winner you have been specially"
    " selected to receive $1000 cash or a $2000 award."
)

print(classify_review(
    text_1, model, tokenizer, device, max_length=train_dataset.max_length
))

text_2 = (
    "Hey, just wanted to check if we're still on"
    " for dinner tonight? Let me know!"
)

print(classify_review(
    text_2, model, tokenizer, device, max_length=train_dataset.max_length
))